# Introduction and main quantities

The quantities are expressed in the following units:
-   density in $10^{19}$ $m^{-3}$
-   temperature in $keV$
-   current in $MA$ 
-   power in $MW$
-   distances in $m$
-   magnetic field in $T$
-   time in $s$

Reminder:
- safety factor at $r=a$: $q_a$
- inverse of the aspect ratio: $\varepsilon=R/a$
- elongation: $\kappa=b/a$
- triangularity: $\delta$


# Governing Equations and Scaling Laws

In [1]:
%matplotlib widget

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.constants import pi, e, k, mu_0

In [3]:
from scaling_laws import *

C_loss=0.09487709734949361
C_fus=0.0016410733400046324
C_beta=0.8053418153388281
C_n=3.183098861837907
C_I=4.999999997278121


Here are the main constants $C_{loss}, C_{fus}, C_\beta, C_n$ and $C_I$

In [4]:
print(f'C_loss={C_loss}')  # OK
print(f'C_fus={C_fus}')  # OK
print(f'C_beta={C_beta}')  # OK
print(f'C_n={C_n}')  # OK
print(f'C_I={C_I}')  # OK

C_loss=0.0815943037205645
C_fus=0.0010082754600988467
C_beta=0.6925939611913922
C_n=3.183098861837907
C_I=13.148529471799288


Let's use additional definition used by [Johner] 

In [5]:
print(f'Meff={Meff}')
print(f'C_loss={C_loss}')
print(f'C_fus={C_fus}')
print(f'C_beta={C_beta}')
print(f'C_I={C_I}') 

Meff=2.6666666666666665
C_loss=0.0815943037205645
C_fus=0.0010082754600988467
C_beta=0.6925939611913922
C_I=13.148529471799288


The triple product $n \times T \times \tau_e$ as a function of $Q$:

In [6]:
# check from §2.8
nTtau_fromQ(Q=10)  # ~300 for Q --> +oo

267.58206128555094

In [7]:
_B = np.linspace(3, 9, 101)
_R = np.linspace(2, 10, 101)
_RR, _BB = np.meshgrid(_R, _B)

_beta_N1 = beta_N1(B=_BB, R=_RR)
_beta_N2 = beta_N2(B=_BB, R=_RR)

In [8]:
fig, ax = plt.subplots(figsize=(7,5))
c1=ax.contour(_RR, _BB, _beta_N1, levels=[1.4, 1.6, 1.8,  2, 2.2],
             alpha=0.5, cmap='copper')
ax.clabel(c1, inline=1, fontsize=10, fmt='%.1f')
c2=ax.contour(_RR, _BB, _beta_N2, levels=[1.4, 1.6,  1.8,  2, 2.2], 
              linestyles='dashed', alpha=0.8)
ax.clabel(c2, inline=1, fontsize=10, fmt='%.1f')
ax.set_xlabel('R [m]')
ax.set_ylabel('B [T]')
ax.set_title(r'plain lines: $\beta_N(R,B)$ from (2.10)'+' \n'+ r' dashed lines: $\beta_N(R,B)$ from (2.20)')

# determine the points where both beta_N are equals to the target (common points)
# from the contour lines
# beta_N_target = 1.8 -> 3rd line on contour
RB_sol1 = c1.allsegs[2][0]
RB_sol2 = c2.allsegs[2][0]
B_sol1 = RB_sol1[:,1]
B_sol2 = np.interp(RB_sol1[:,0], RB_sol2[:,0], RB_sol2[:,1])  # interpolate to be able to find the nearest value 

R_sol = RB_sol1[np.argmin(np.abs(B_sol1 - B_sol2)), 0]
B_sol = RB_sol1[np.argmin(np.abs(B_sol1 - B_sol2)), 1]

ax.plot(R_sol, B_sol, '.', ms=20, color='k')

ax.annotate(fr'(R,B) for $\beta_N=1.8$: ({R_sol:.2f} m, {B_sol:.2f} T)', 
            xy=(R_sol+.1, B_sol+.1), xytext=(5,8),
            arrowprops={'arrowstyle': '->'}, va='center')

fig.savefig('beta_N.png', dpi=150)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Since there is two equations $\beta_{N,1}(R,B)$ and $\beta_{N,2}(R,B)$, we solve for $(R,B)$ 

In [9]:
# for each R, find the points B for which beta_N1 = beta_N2
from scipy.optimize import fsolve

def diff_beta_N12(B, R):
    return beta_N1(R=R, B=B) - beta_N2(R=R, B=B)

B_sol12 = []
B0 = 5
for R in _R:
    B_sol12.append(fsolve(diff_beta_N12, B0, args=R))
B_sol12 = np.array(B_sol12).squeeze()

In [10]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
c=ax.plot_surface(_RR, _BB, _beta_N1, alpha=0.6, cmap=cm.viridis, vmin=0, vmax=3)
ax.plot_surface(_RR, _BB, _beta_N2, alpha=0.6, cmap=cm.viridis, vmin=0, vmax=3)
ax.set_xlabel('R [m]')
ax.set_ylabel('B [T]')
ax.set_zlabel(r'$\beta_N$')
#ax.set_aspect('equal')
ax.set_zlim(0, 5)
ax.plot(_R, B_sol12, beta_N1(R=_R, B=B_sol12), color='r', lw=2)
ax.set_xlim(2, 10)
ax.set_ylim(3, 9)
ax.view_init(azim=-45, elev=25)
fig.colorbar(c, shrink=0.7)
fig.tight_layout()
ax.set_title(r'$\beta_N$(R,B) from eq.(2.10) and (2.20)')
fig.savefig('beta_N_3D.png', dpi=150)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
# relative distance between the two surfaces
delta_beta_N12 = 2*np.abs(_beta_N1 - _beta_N2)/(_beta_N1 + _beta_N2)

In [12]:
fig, ax = plt.subplots()
c=ax.pcolor(_R, _B, diff_beta_N12(_BB, _RR), cmap='RdGy', vmin=-10, vmax=10)
fig.colorbar(c)
ax.set_ylim(3,9)
ax.set_xlim(2, 10)
ax.plot(_R, B_sol12, lw=3)
#ax.plot(_R, B_sol13)
ax.set_xlabel('R [m]')
ax.set_ylabel('$B_0$ [T]')
ax.set_title(r'$\Delta\beta_N(R,B_0)$')
fig.savefig('Beta_N_2D.png', dpi=150)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\JH218595\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  


# Retrieving ITER Parameters

# Solving from minimization

## Solving for $\beta_N(R,B)$

In [13]:
# return the distance between between the target beta_N and the beta_N functions of (R,B)
# this function is minimized to find the optimal (R,B) to get the prescribed beta_N
def obj_fun_beta(x, beta_N=1.8):  
    # parameters to find
    R, B = x 
    
    _beta_N1 = beta_N1(R=R, B=B)
    _beta_N2 = beta_N2(R=R, B=B)

    # return scalar objective function
    return np.sqrt((beta_N - _beta_N1)**2 + (beta_N - _beta_N2)**2)

In [14]:
R0, B0 = 5, 4
x0 = [R0, B0]

x = minimize(obj_fun_beta, x0, args=[1.8])
print(f'Solution found: R={x.x[0]:.2f} m, B={x.x[1]:.2f} T')

Solution found: R=4.94 m, B=6.12 T


## Stylin'

In [15]:
from IPython.core.display import HTML

def _set_css_style(css_file_path):
    styles = open(css_file_path, 'r').read()
    s = '<style>%s</style>' % styles
    return HTML(s)
_set_css_style('../custom.css')